<a href="https://colab.research.google.com/github/PinkPigmyPuff/SeatingChart/blob/main/ClassroomGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import random
import numpy as np
from typing import List, Optional, Callable, Tuple


Chromosome = List[int]
Population = List[Chromosome]
Opinion = List[int]

peoplePerTable = 4
nextToFriendCost = 10
nextToEnemyCost = -.01
multipleFriendCost = -.01

In [30]:
def genChromosome(size: int) -> Chromosome:
    return random.sample(range(0, size), size)

def genPopulation(popSize: int, chromeosomeSize: int) -> Population:
    return [genChromosome(chromeosomeSize) for _ in range(popSize)]

def genOpinions(chromosome: Chromosome, opinionMax: int) -> Opinion:
  likes = []
  dislikes = []
  # for every student, randomly generate how many students they have opionions on, and add an empty array to like/dislike
  for i in range(0, len(chromosome)):
    likes.append([])
    dislikes.append([])
    likeNum = random.randint(0, opinionMax)
    dislikeNum = random.randint(0, opinionMax)
    dupelist = chromosome.copy()
    dupelist.pop(i)
    
    # choose random students that the current student likes
    for j in range(0, likeNum):
      randomStudent = random.randrange(0, len(dupelist))
      likes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

    # choose random students that the current student dislikes
    for j in range(0, dislikeNum):
      randomStudent = random.randrange(0, len(dupelist))
      dislikes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

  return likes, dislikes

In [31]:
def alleleFitness(chromosome: Chromosome, allele: int) -> float:
      currentStudent = chromosome[allele]
      studentsTable = findTable(chromosome, currentStudent)
      friendNum = 0
      alleleHappiness = 0
        # print(f"current student is {currentStudent}, and they sit at table {studentsTable}")

      # determine the happiness of the currentStudent, based on each person at their table
      for tableMate in range(0, peoplePerTable):
        currentTableMate = chromosome[studentsTable][tableMate]
        if currentTableMate in likes[allele]:
          friendNum += 1
          if friendNum == 1:
            alleleHappiness += nextToFriendCost
          if friendNum >= 2:
            alleleHappiness += multipleFriendCost

        elif currentTableMate in dislikes[allele]:
          alleleHappiness += nextToEnemyCost

        print(f"Student {currentStudent} happiness: {alleleHappiness}")
        return alleleHappiness

def fitness(chromosome: Chromosome) -> int:
    size = len(chromosome)
    if size != len(likes) != len(dislikes):
        raise ValueError("genome and likes/dislikes must be of the same length")

    if size % peoplePerTable != 0:
        raise ValueError("The length of the chromosome must be divisable by peoplePerTable")

    # determine the cost of the given chromosome
    peopleHappiness = [0] * size
    chunkedClassroom = chunks(chromosome, peoplePerTable)

    # for every student
    for student in range(0, len(chromosome)):
      peopleHappiness[student] = alleleFitness(chunkedClassroom, student)

    return peopleHappiness

def populationFitness(population: Population) -> float:
    return [average(fitness(chromosome)) for chromosome in population]
    
# Return n-sized chunks from lst
def chunks(lst, n):
    x = [lst[i:i + n] for i in range(0, len(lst), n)]
    return(x)

def relu(x):
	  return max(0.0, x)
 
def average(lst):
   return float(sum(lst) / len(lst))

In [32]:
# finds which table the currentStudent is sitting at
def findTable(classroom, student):
      if any(student in (match := nested_list) for nested_list in classroom):
        return classroom.index(match)

In [33]:
# Randomly chooses n individuals from the population and returns the fittest one
def tournamentSelection(population: Population, tournamentSize: int) -> Population:
		
		tournament = []
		tempPop = population.copy()
		for i in range(0, tournamentSize):
			random_id = random.randint(0, len(tempPop)-1)
			tournament.append(tempPop[random_id])
			tempPop.pop(i)

		#print(f"tournament pool! {tournament}")
		fittest = chooseHighest(tournament)
		#print(f"fittest! {fittest}")
		return fittest

In [34]:
def roulette_wheel_selection(population: Population, populationFitness) -> Population:
    if len(population) != len(populationFitness):
        raise ValueError("genome and likes/dislikes must be of the same length")
    print(f"pFit: {populationFitness}")

    # Computes the totallity of the population fitness
    sumFitness = sum(populationFitness)
    print(f"sumFittness: {sumFitness}")
    
    # Computes for each chromosome the probability 
    chromosome_probabilities = [classroomHappiness/sumFitness for classroomHappiness in populationFitness]
    print(f"c_prob {chromosome_probabilities}")

    # Selects one chromosome based on the computed probabilities
    choice = population[np.random.choice(range(0, len(population)), p=chromosome_probabilities)]
    print(f"chioce_fitness {average(fitness(choice))}")
    return choice

In [35]:
def partiallyMatchedCrossover(ind1, ind2):
    #print(f"ind1: {ind1}\nind2: {ind2}")
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    #print(f"FINSIHED, ind1: {ind1}\n ind2: {ind2}")
    return ind1, ind2

In [36]:
def mutate(chromosome: Chromosome) -> Chromosome:
    length = len(chromosome)
    if length < 2:
      print("Chromosome is too short to be mutated")
      return chromosome
    index1, index2 = random.sample(range(0, length), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    #print(f"mutated chromosome: {chromosome}")
    return chromosome

In [37]:
def chooseHighest(population: Population) -> Chromosome:
  #print(f"pop to choose from: {population}")
  fit = populationFitness(population)
  #print(f"pop fitness: {fit}")
  highestIndex = 0
  for i in range(0, len(fit)):
      if fit[i] > fit[highestIndex]:
          highest = fit[i]
          highestIndex = i
  #print(f"chosen: {population[highestIndex]}")
  return population[highestIndex]

In [38]:
def studentInfo(chromosome: Chromosome):
  for i in range(len(chromosome)):
    chunkedClassroom = chunks(chromosome, 4)
          # determine the happiness of the currentStudent, based on each person at their table
    studenTable = findTable(chunkedClassroom, i)
    print(f"\n\nstudent {i} likes: {likes[i]}, dislikes: {dislikes[i]}, is sitting at table: {studenTable}: ", end="")
    for tableMate in range(0, 4):
        currentTableMate = chunkedClassroom[studenTable][tableMate]
        print(f"{currentTableMate}", end=", ")

In [39]:
pop = genPopulation(40, 24)
likes, dislikes = genOpinions(pop[0], 4)

def run_evolution(
        generation_limit: int = 100,
        population = genPopulation(8, 8), 
        #likes = genOpinions(population[0], 4),
        #dislikes = genOpinions(population[0], 4)
        #peoplePerTable = 4,
        #nextToFriendCost = 1,
        #nextToEnemyCost = -1,
        ):
  
    print(f"OG population: {population}")
    print(f"Likes: {likes}\nDislikes: {dislikes}")
    if len(population) % 2 != 0:
        print("Population size is odd, removing one chromosome")
        population.pop(0)


    for i in range(generation_limit):
        nextGeneration = []

        # Find and print the fitness of the population, for Debug reasons
        popFitness = populationFitness(population)
        print(f"\n\nGeneration {i} avg fitness: {average(popFitness)}, {popFitness}")
        print(f"population: {population}")

        # Create a new parent population, by selecting the most fit members of the population
        parents = [tournamentSelection(population, 4) for i in range(len(population))]
        print(f"parents: {parents}")

        # Have one pair of parents breed at a time; then mutate their children, and remove the parents from the breeding pool
        while parents:
          #print(f"b0: {parents[0]}\nb1: {parents[1]}")
          offspring1, offspring2 = partiallyMatchedCrossover(parents[0].copy(), parents[1].copy())
          nextGeneration.append(mutate(offspring1.copy()))
          nextGeneration.append(mutate(offspring2.copy()))          
          parents = parents[2:]

        # Set the current population to equal the kids
        population = nextGeneration.copy()
    
    return chooseHighest(population)


highest = run_evolution(population=pop)
print(highest)
print(fitness(highest))
print(studentInfo(highest))

OG population: [[17, 12, 16, 18, 8, 6, 2, 15, 22, 11, 20, 9, 14, 5, 10, 13, 0, 4, 21, 1, 19, 3, 23, 7], [1, 12, 10, 16, 23, 3, 9, 13, 17, 4, 21, 22, 20, 18, 14, 0, 7, 2, 8, 15, 11, 19, 6, 5], [8, 13, 17, 11, 16, 19, 15, 7, 20, 5, 4, 9, 1, 22, 21, 14, 3, 23, 2, 6, 0, 10, 18, 12], [14, 21, 0, 18, 16, 2, 17, 4, 19, 23, 9, 8, 12, 1, 3, 7, 11, 15, 13, 20, 10, 5, 22, 6], [20, 3, 21, 10, 7, 4, 2, 22, 9, 6, 8, 1, 5, 14, 15, 23, 13, 17, 19, 12, 18, 11, 0, 16], [5, 11, 20, 23, 13, 10, 16, 4, 19, 2, 0, 1, 3, 21, 9, 22, 14, 8, 6, 17, 15, 12, 18, 7], [23, 10, 21, 5, 8, 17, 14, 2, 1, 13, 11, 3, 0, 19, 4, 15, 6, 20, 22, 12, 9, 16, 18, 7], [11, 8, 16, 4, 6, 14, 22, 13, 3, 9, 7, 12, 15, 21, 10, 0, 2, 1, 19, 20, 23, 17, 18, 5], [23, 5, 19, 8, 22, 20, 6, 15, 7, 17, 3, 10, 13, 21, 9, 16, 14, 0, 1, 12, 2, 18, 11, 4], [18, 4, 20, 13, 2, 7, 5, 12, 0, 16, 1, 6, 3, 8, 14, 11, 15, 10, 19, 21, 23, 9, 22, 17], [8, 16, 19, 2, 21, 15, 9, 10, 20, 7, 0, 18, 6, 12, 5, 22, 23, 3, 17, 14, 11, 4, 1, 13], [16, 17, 14, 19,

TypeError: ignored

In [ ]:
# Test hundreds of scenarios to determine avg. success
def test(n):
    pop = genPopulation(40, 20)
    likes, dislikes = genOpinions(pop[0], 3)

    for i in range(n):
        run_evolution(population=pop)

1. Encoding scheme
- A chromosome is a permutation list
- Each index is an integer
- Each integer is a student in the class

2. Create the initial population
- First, a function to create a single chromsome
  - inputs chromosomeSize
  - outputs a chromosome of randomly shuffled integers

7. Testing
- A func

Todo:

- Add custom classroom layouts, for example with variable table sizes. Could be done w/ an empty, nested list to represent classroom.
- Add a CSV -> Data translater
- Move out of Google Colab, and into modular scripts (after implementing CSV, etc)
- Add a "special requests" part to the fitness function, such as needs to be near the front of the class
https://github.com/DEAP/deap/blob/master/deap/tools/ 

In [ ]:
import random
import warnings

try:
    from collections.abc import Sequence
except ImportError:
    from collections import Sequence

from itertools import repeat


######################################
# GA Crossovers                      #
######################################


def cxOnePoint(ind1, ind2):
    """Executes a one point crossover on the input :term:`sequence` individuals.
    The two individuals are modified in place. The resulting individuals will
    respectively have the length of the other.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the
    python base :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size - 1)
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    return ind1, ind2


def cxTwoPoint(ind1, ind2):
    """Executes a two-point crossover on the input :term:`sequence`
    individuals. The two individuals are modified in place and both keep
    their original length.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the Python
    base :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2


def cxTwoPoints(ind1, ind2):
    """
    .. deprecated:: 1.0
       The function has been renamed.  Use :func:`~deap.tools.cxTwoPoint` instead.
    """
    warnings.warn("tools.cxTwoPoints has been renamed. Use cxTwoPoint instead.",
                  FutureWarning)
    return cxTwoPoint(ind1, ind2)


def cxUniform(ind1, ind2, indpb):
    """Executes a uniform crossover that modify in place the two
    :term:`sequence` individuals. The attributes are swapped according to the
    *indpb* probability.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :param indpb: Independent probability for each attribute to be exchanged.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))
    for i in range(size):
        if random.random() < indpb:
            ind1[i], ind2[i] = ind2[i], ind1[i]

    return ind1, ind2


def cxPartialyMatched(ind1, ind2):
    """Executes a partially matched crossover (PMX) on the input individuals.
    The two individuals are modified in place. This crossover expects
    :term:`sequence` individuals of indices, the result for any other type of
    individuals is unpredictable.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    Moreover, this crossover generates two children by matching
    pairs of values in a certain range of the two parents and swapping the values
    of those indexes. For more details see [Goldberg1985]_.
    This function uses the :func:`~random.randint` function from the python base
    :mod:`random` module.
    .. [Goldberg1985] Goldberg and Lingel, "Alleles, loci, and the traveling
       salesman problem", 1985.
    """
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2


def cxUniformPartialyMatched(ind1, ind2, indpb):
    """Executes a uniform partially matched crossover (UPMX) on the input
    individuals. The two individuals are modified in place. This crossover
    expects :term:`sequence` individuals of indices, the result for any other
    type of individuals is unpredictable.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    Moreover, this crossover generates two children by matching
    pairs of values chosen at random with a probability of *indpb* in the two
    parents and swapping the values of those indexes. For more details see
    [Cicirello2000]_.
    This function uses the :func:`~random.random` and :func:`~random.randint`
    functions from the python base :mod:`random` module.
    .. [Cicirello2000] Cicirello and Smith, "Modeling GA performance for
       control parameter optimization", 2000.
    """
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i

    for i in range(size):
        if random.random() < indpb:
            # Keep track of the selected values
            temp1 = ind1[i]
            temp2 = ind2[i]
            # Swap the matched value
            ind1[i], ind1[p1[temp2]] = temp2, temp1
            ind2[i], ind2[p2[temp1]] = temp1, temp2
            # Position bookkeeping
            p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
            p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2


def cxOrdered(ind1, ind2):
    """Executes an ordered crossover (OX) on the input
    individuals. The two individuals are modified in place. This crossover
    expects :term:`sequence` individuals of indices, the result for any other
    type of individuals is unpredictable.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    Moreover, this crossover generates holes in the input
    individuals. A hole is created when an attribute of an individual is
    between the two crossover points of the other individual. Then it rotates
    the element so that all holes are between the crossover points and fills
    them with the removed elements in order. For more details see
    [Goldberg1989]_.
    This function uses the :func:`~random.sample` function from the python base
    :mod:`random` module.
    .. [Goldberg1989] Goldberg. Genetic algorithms in search,
       optimization and machine learning. Addison Wesley, 1989
    """
    size = min(len(ind1), len(ind2))
    a, b = random.sample(range(size), 2)
    if a > b:
        a, b = b, a

    holes1, holes2 = [True] * size, [True] * size
    for i in range(size):
        if i < a or i > b:
            holes1[ind2[i]] = False
            holes2[ind1[i]] = False

    # We must keep the original values somewhere before scrambling everything
    temp1, temp2 = ind1, ind2
    k1, k2 = b + 1, b + 1
    for i in range(size):
        if not holes1[temp1[(i + b + 1) % size]]:
            ind1[k1 % size] = temp1[(i + b + 1) % size]
            k1 += 1

        if not holes2[temp2[(i + b + 1) % size]]:
            ind2[k2 % size] = temp2[(i + b + 1) % size]
            k2 += 1

    # Swap the content between a and b (included)
    for i in range(a, b + 1):
        ind1[i], ind2[i] = ind2[i], ind1[i]

    return ind1, ind2


def cxBlend(ind1, ind2, alpha):
    """Executes a blend crossover that modify in-place the input individuals.
    The blend crossover expects :term:`sequence` individuals of floating point
    numbers.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :param alpha: Extent of the interval in which the new values can be drawn
                  for each attribute on both side of the parents' attributes.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    for i, (x1, x2) in enumerate(zip(ind1, ind2)):
        gamma = (1. + 2. * alpha) * random.random() - alpha
        ind1[i] = (1. - gamma) * x1 + gamma * x2
        ind2[i] = gamma * x1 + (1. - gamma) * x2

    return ind1, ind2


def cxSimulatedBinary(ind1, ind2, eta):
    """Executes a simulated binary crossover that modify in-place the input
    individuals. The simulated binary crossover expects :term:`sequence`
    individuals of floating point numbers.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :param eta: Crowding degree of the crossover. A high eta will produce
                children resembling to their parents, while a small eta will
                produce solutions much more different.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    for i, (x1, x2) in enumerate(zip(ind1, ind2)):
        rand = random.random()
        if rand <= 0.5:
            beta = 2. * rand
        else:
            beta = 1. / (2. * (1. - rand))
        beta **= 1. / (eta + 1.)
        ind1[i] = 0.5 * (((1 + beta) * x1) + ((1 - beta) * x2))
        ind2[i] = 0.5 * (((1 - beta) * x1) + ((1 + beta) * x2))

    return ind1, ind2


def cxSimulatedBinaryBounded(ind1, ind2, eta, low, up):
    """Executes a simulated binary crossover that modify in-place the input
    individuals. The simulated binary crossover expects :term:`sequence`
    individuals of floating point numbers.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :param eta: Crowding degree of the crossover. A high eta will produce
                children resembling to their parents, while a small eta will
                produce solutions much more different.
    :param low: A value or a :term:`python:sequence` of values that is the lower
                bound of the search space.
    :param up: A value or a :term:`python:sequence` of values that is the upper
               bound of the search space.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    .. note::
       This implementation is similar to the one implemented in the
       original NSGA-II C code presented by Deb.
    """
    size = min(len(ind1), len(ind2))
    if not isinstance(low, Sequence):
        low = repeat(low, size)
    elif len(low) < size:
        raise IndexError("low must be at least the size of the shorter individual: %d < %d" % (len(low), size))
    if not isinstance(up, Sequence):
        up = repeat(up, size)
    elif len(up) < size:
        raise IndexError("up must be at least the size of the shorter individual: %d < %d" % (len(up), size))

    for i, xl, xu in zip(range(size), low, up):
        if random.random() <= 0.5:
            # This epsilon should probably be changed for 0 since
            # floating point arithmetic in Python is safer
            if abs(ind1[i] - ind2[i]) > 1e-14:
                x1 = min(ind1[i], ind2[i])
                x2 = max(ind1[i], ind2[i])
                rand = random.random()

                beta = 1.0 + (2.0 * (x1 - xl) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                if rand <= 1.0 / alpha:
                    beta_q = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    beta_q = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))

                c1 = 0.5 * (x1 + x2 - beta_q * (x2 - x1))

                beta = 1.0 + (2.0 * (xu - x2) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                if rand <= 1.0 / alpha:
                    beta_q = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    beta_q = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))
                c2 = 0.5 * (x1 + x2 + beta_q * (x2 - x1))

                c1 = min(max(c1, xl), xu)
                c2 = min(max(c2, xl), xu)

                if random.random() <= 0.5:
                    ind1[i] = c2
                    ind2[i] = c1
                else:
                    ind1[i] = c1
                    ind2[i] = c2

    return ind1, ind2


######################################
# Messy Crossovers                   #
######################################

def cxMessyOnePoint(ind1, ind2):
    """Executes a one point crossover on :term:`sequence` individual.
    The crossover will in most cases change the individuals size. The two
    individuals are modified in place.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the python base
    :mod:`random` module.
    """
    cxpoint1 = random.randint(0, len(ind1))
    cxpoint2 = random.randint(0, len(ind2))
    ind1[cxpoint1:], ind2[cxpoint2:] = ind2[cxpoint2:], ind1[cxpoint1:]

    return ind1, ind2


######################################
# ES Crossovers                      #
######################################

def cxESBlend(ind1, ind2, alpha):
    """Executes a blend crossover on both, the individual and the strategy. The
    individuals shall be a :term:`sequence` and must have a :term:`sequence`
    :attr:`strategy` attribute. Adjustment of the minimal strategy shall be done
    after the call to this function, consider using a decorator.
    :param ind1: The first evolution strategy participating in the crossover.
    :param ind2: The second evolution strategy participating in the crossover.
    :param alpha: Extent of the interval in which the new values can be drawn
                  for each attribute on both side of the parents' attributes.
    :returns: A tuple of two evolution strategies.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    for i, (x1, s1, x2, s2) in enumerate(zip(ind1, ind1.strategy,
                                             ind2, ind2.strategy)):
        # Blend the values
        gamma = (1. + 2. * alpha) * random.random() - alpha
        ind1[i] = (1. - gamma) * x1 + gamma * x2
        ind2[i] = gamma * x1 + (1. - gamma) * x2
        # Blend the strategies
        gamma = (1. + 2. * alpha) * random.random() - alpha
        ind1.strategy[i] = (1. - gamma) * s1 + gamma * s2
        ind2.strategy[i] = gamma * s1 + (1. - gamma) * s2

    return ind1, ind2


def cxESTwoPoint(ind1, ind2):
    """Executes a classical two points crossover on both the individuals and their
    strategy. The individuals shall be a :term:`sequence` and must have a
    :term:`sequence` :attr:`strategy` attribute. The crossover points for the
    individual and the strategy are the same.
    :param ind1: The first evolution strategy participating in the crossover.
    :param ind2: The second evolution strategy participating in the crossover.
    :returns: A tuple of two evolution strategies.
    This function uses the :func:`~random.randint` function from the python base
    :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))

    pt1 = random.randint(1, size)
    pt2 = random.randint(1, size - 1)
    if pt2 >= pt1:
        pt2 += 1
    else:  # Swap the two cx points
        pt1, pt2 = pt2, pt1

    ind1[pt1:pt2], ind2[pt1:pt2] = ind2[pt1:pt2], ind1[pt1:pt2]
    ind1.strategy[pt1:pt2], ind2.strategy[pt1:pt2] = \
        ind2.strategy[pt1:pt2], ind1.strategy[pt1:pt2]

    return ind1, ind2


def cxESTwoPoints(ind1, ind2):
    """
    .. deprecated:: 1.0
        The function has been renamed. Use :func:`cxESTwoPoint` instead.
    """
    return cxESTwoPoint(ind1, ind2)


# List of exported function names.
__all__ = ['cxOnePoint', 'cxTwoPoint', 'cxUniform', 'cxPartialyMatched',
           'cxUniformPartialyMatched', 'cxOrdered', 'cxBlend',
           'cxSimulatedBinary', 'cxSimulatedBinaryBounded', 'cxMessyOnePoint',
           'cxESBlend', 'cxESTwoPoint']